In [10]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
import numpy as np
import os,random, math
from tqdm import tqdm
from copy import deepcopy
from collections import Counter

# Visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'NanumGothic'
import platform
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

# from dataprep.eda import plot, plot_correlation, plot_missing

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [11]:
import os
path = os.path.dirname(os.getcwd())
path = os.path.join(path, "ProcessedData")
train = pd.read_csv(os.path.join(path,"merged_train.csv"))
test = pd.read_csv(os.path.join(path,"merged_test.csv"))

In [12]:
apt_test = test[test['임대건물구분']=='아파트']
store_test = test[test['임대건물구분']=='상가']

In [13]:
drop = ['남/여비율','남/여_0~19세','남/여_20~39세','남/여_40~69세','남/여_70세이상','0~19세_비율','20~39세_비율','40~69세_비율','70세이상_비율']
drop1 = ['총세대수','임대건물구분','지역','공가수','자격유형','단지내주차면수','단지명','도로명주소','subway_name']

train = train.drop(drop,axis=1)
test = test.drop(drop,axis=1)

train = train.drop(drop1,axis=1)
test = test.drop(drop1,axis=1)

In [14]:
non_unique = ['전용면적','전용면적별세대수','임대보증금','임대료']
train_one = train.drop(non_unique,axis=1).drop_duplicates().reset_index(drop=True)
test_one = test.drop(non_unique,axis=1).drop_duplicates().reset_index(drop=True)

In [15]:
def merge(col,standard,oldDF,newDF,kind):
    sample = oldDF.groupby('단지코드').describe()[col].reset_index()
    for i in sample.index:
        for j in standard:
            code = sample.loc[i,'단지코드']
            val = sample.loc[i,j]
            ind = newDF[newDF['단지코드']==code].index
            newDF.loc[ind,kind+col+j] = val
    return newDF

In [17]:
test_one = merge('임대료',['min','25%','50%','75%','max','mean','std','count'],apt_test,test_one,'apt')
test_one = merge('임대보증금',['min','25%','50%','75%','max','mean','std','count'],apt_test,test_one,'apt')

test_one = merge('임대료',['min','25%','50%','75%','max','mean','std','count'],store_test,test_one,'store')
test_one = merge('임대보증금',['min','25%','50%','75%','max','mean','std','count'],store_test,test_one,'store')

test_one = merge('전용면적',['min','25%','50%','75%','max','mean','std','count'],apt_test,test_one,'apt')
test_one = merge('전용면적',['min','25%','50%','75%','max','mean','std','count'],store_test,test_one,'store')

test_one = test_one.replace(np.nan, 0)

In [ ]:
test_one.to_csv("data/test_merged.csv")